<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
from torch import nn
import torch.onnx
from openvino.runtime import Core
from openvino.tools import mo
import os

class FFDNet(nn.Module):
    def __init__(self, num_channels=1, num_features=64):
        super(FFDNet, self).__init__()
        self.num_channels = num_channels
        self.num_features = num_features

        # 다운샘플링
        self.downscale = nn.PixelUnshuffle(2)

        # 특징 추출 레이어
        layers = []
        # 첫 번째 컨볼루션 레이어
        layers.append(nn.Conv2d(num_channels * 4 + 1, num_features, kernel_size=3, padding=1))
        layers.append(nn.ReLU(inplace=True))

        # 중간 레이어
        for _ in range(12):  # 12개의 중간 레이어
            layers.append(nn.Conv2d(num_features, num_features, kernel_size=3, padding=1))
            layers.append(nn.BatchNorm2d(num_features))
            layers.append(nn.ReLU(inplace=True))

        # 마지막 컨볼루션 레이어
        layers.append(nn.Conv2d(num_features, num_channels * 4, kernel_size=3, padding=1))

        self.features = nn.Sequential(*layers)

        # 업샘플링
        self.upscale = nn.PixelShuffle(2)

    def forward(self, x, noise_level):
        # 다운샘플링
        x_down = self.downscale(x)

        # 노이즈 레벨 맵 생성 및 확장
        noise_level = noise_level.expand(-1, -1, x_down.shape[2], x_down.shape[3])

        # 입력과 노이즈 레벨 맵 결합
        x_in = torch.cat([x_down, noise_level], dim=1)

        # 특징 추출
        features = self.features(x_in)

        # 업샘플링
        out = self.upscale(features)

        return out

def create_noise_level_map(noise_sigma, shape):
    """노이즈 레벨 맵 생성 함수"""
    return torch.full((1, 1, shape[2], shape[3]), noise_sigma)

def convert_to_openvino(model_path, output_dir):
    """PyTorch 모델을 OpenVINO 모델로 변환하는 함수"""
    try:
        os.makedirs(output_dir, exist_ok=True)

        # 모델 로드
        model = FFDNet(num_channels=1)
        model.load_state_dict(torch.load(model_path))
        model.eval()

        # ONNX 모델 경로
        onnx_path = os.path.join(output_dir, "ffdnet.onnx")

        # 입력 샘플 생성
        sample_image = torch.randn(1, 1, 256, 256)
        sample_noise = torch.full((1, 1, 1, 1), 0.1)

        # 추가된 레이어의 중간 출력을 추적하기 위한 설정
        class ModelWrapper(nn.Module):
            def __init__(self, model):
                super(ModelWrapper, self).__init__()
                self.model = model

            def forward(self, x, noise):
                # 중간 출력을 저장할 딕셔너리
                intermediate = {}

                # 다운샘플링
                x_down = self.model.downscale(x)
                intermediate['downscale'] = x_down

                # 노이즈 레벨 맵
                noise_map = noise.expand(-1, -1, x_down.shape[2], x_down.shape[3])
                x_in = torch.cat([x_down, noise_map], dim=1)

                # 특징 추출
                features = self.model.features(x_in)
                intermediate['features'] = features

                # 업샘플링
                out = self.model.upscale(features)
                intermediate['upscale'] = out

                return out, intermediate

        # 모델 래핑
        wrapped_model = ModelWrapper(model)

        # PyTorch -> ONNX 변환
        print("PyTorch 모델을 ONNX로 변환 중...")
        torch.onnx.export(wrapped_model,
                         (sample_image, sample_noise),
                         onnx_path,
                         input_names=['input_image', 'noise_level'],
                         output_names=['denoised_image', 'intermediate_outputs'],
                         dynamic_axes={
                             'input_image': {0: 'batch_size', 2: 'height', 3: 'width'},
                             'noise_level': {0: 'batch_size'},
                             'denoised_image': {0: 'batch_size', 2: 'height', 3: 'width'}
                         },
                         do_constant_folding=True,
                         opset_version=11)

        print("ONNX 모델 저장됨:", onnx_path)

        # ONNX -> OpenVINO IR 변환
        print("ONNX 모델을 OpenVINO로 변환 중...")
        ov_model = mo.convert_model(
            onnx_path,
            model_name="ffdnet",
            framework="onnx",
            input=['input_image', 'noise_level'],
            input_shape=[[1, 1, 256, 256], [1, 1, 1, 1]],
            output=['denoised_image', 'intermediate_outputs'],
            data_type='FP32'
        )

        # OpenVINO 모델 저장
        serialize_path = os.path.join(output_dir, "ffdnet.xml")
        ov_model.serialize(serialize_path)

        print("변환 완료")

        # 변환된 모델 테스트
        print("변환된 모델 테스트 중...")
        ie = Core()
        model = ie.read_model(serialize_path)
        compiled_model = ie.compile_model(model=model, device_name="CPU")

        # 입력 레이어 정보 출력
        print("\n입력 레이어 정보:")
        for input_layer in model.inputs:
            print(f"이름: {input_layer.get_any_name()}")
            print(f"형상: {input_layer.shape}")
            print(f"타입: {input_layer.element_type}")
            print("---")

        # 출력 레이어 정보 출력
        print("\n출력 레이어 정보:")
        for output_layer in model.outputs:
            print(f"이름: {output_layer.get_any_name()}")
            print(f"형상: {output_layer.shape}")
            print(f"타입: {output_layer.element_type}")
            print("---")

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        raise

if __name__ == "__main__":
    model_path = "path/to/ffdnet.pth"
    output_dir = "openvino_model"
    convert_to_openvino(model_path, output_dir)